1. Bibliografía:
  - El **tema 6** permitirá al alumno familiarizarse con la programación genética (PG) 
  - y, **el mencionado artículo**, con una de las variantes de la PG, denominada Evolución Gramatical (del inglés Grammatical Evolution, GE). 
  - El **capítulo 8** describe distintos mecanismos para sintonizar de forma adaptativa cada uno de los diferentes parámetros de los que consta un algoritmo evolutivo (AE). 
  - El **capítulo 10** describe la forma de hibridar un AE con otros métodos de búsqueda. 
  - Finalmente, en el **capítulo 12**, se muestran distintas estrategias para manejar la existencia de restricciones en problemas de optimización que son abordados mediante AEs.
  
2. Secciones
  - Descripción del problema a resolver
  - Método para resolverlo 
    - se debe analizar la idoneidad o no del uso de GE para resolver el problema planteado
    - se debe incluir la expresión matemática de la función de evaluación finalmente empleada
    - se debe incluir la descripción de los diferentes operadores de inicialización, variación y selección empleados
    - se debe incluir la forma de manejar las restricciones, los mecanismos de control de parámetros utilizados, así como los mecanismos de búsqueda local implementados
  - Los resultados de los distintos experimentos realizados
  - Un análisis y comparación de resultados
  - Una sección de conclusiones
  - Una descripción del código implementado. 

3. Evaluación
  - Sobre la presentación (2/10)
    - Se evaluará especialmente la claridad en la redacción de la memoria y la capacidad de síntesis.  
  - Sobre el manejo de restricciones (1/10)
      - Se valorará la originalidad del mecanismo o mecanismos usados para el manejo de restricciones.
  - Sobre la configuración del algoritmo (2/10)
    - Aquí se valorará el procedimiento seguido por el alumno a la hora de elegir la mejor configuración de parámetros del algoritmo, incluyendo la implementación de mecanismos de control de parámetros adaptativos o auto-adaptativos
  - Sobre la hibridación del algoritmo con técnicas de búsqueda local (1/10)
    - Se valorará la originalidad del mecanismo de búsqueda local utilizado.
  - Sobre el análisis y comparación de resultados, y conclusiones (4/10)
    - Se valorará la forma de interpretar y comparar los diferentes experimentos realizados.
      - Es muy importante que dicha valoración se haga siempre en términos de los índices SR, MBF, AES 
      - y cualquier otra gráfica que considere oportuna como, por ejemplo, los plots de progreso de convergencia. 
    - Finalmente, se valorará la calidad de las conclusiones obtenidas a partir de la interpretación y comparación de resultados.


# 1. Descripción del problema a resolver

Repitiendo las indicaciones dadas en el documento de la actividad, el problema consiste en implementar un algoritmo evolutivo para calcular la derivada simbólica de una función 
$$ f:X \subseteq \mathcal{R} \rightarrow \mathcal{R} $$ 

Disponemos de las siguientes dos definiciones:

> **Definición de derivada de una función en un punto**: Sea $X \subseteq \mathcal{R}$ un intervalo abierto. Diremos que $f:X \subseteq \mathcal{R} \rightarrow \mathcal{R}$ es derivable en $x_0 \in X$, denotado por $f'(x_0)$, si existe y es finito el límite:
$$
f'(x_0) = \lim \limits_{h \to 0} \frac{f(x_0+h)-f(x_0)}{h}  \tag{1}
$$

> **Definición de derivada de una función en un intervalo**: Sea $X \subseteq \mathcal{R}$ un intervalo abierto. Diremos que $f:X \subseteq \mathcal{R} \rightarrow \mathcal{R}$ es derivable en el intervalo $[a,b] \subseteq X$, si $f$ es derivable en cada uno de los puntos de dicho intervalo, es decir, si:
$$
f'(x) = \lim \limits_{h \to 0} \frac{f(x+h)-f(x)}{h}, \forall x \in [a,b]  \tag{2}
$$

Suponiendo que $f$ sea derivable en $[a,b]$, el problema de calcular la derivada lo vamos a transformar en un nuevo problema de optimización consistente en encontrar una función $g(x)$ que minimice la expresión:
$$
\min \limits_{g(x)} \frac{1}{b-a}\int_{a}^{b} error[f'(x),g(x)]dx \tag{3}
$$

dónde $f'(x)$ se calcularía utilizando la expresión $(2)$.

No obstante, el problema anterior se puede resolver de forma aproximada discretizando el intervalo de definición, es decir, cambiando el operador integral por un sumatorio:
$$
\min \limits_{g(x)} \frac{1}{N+1}\sum_{i=0}^{N} error_i[f'(a+i*h),g(a+i*h)] \tag{4}
$$
dónde $h=\frac{b-a}{N}$ es la anchura del subintervalo de muestreo para conseguir muestrear $N+1$ puntos en el intervalo $[a,b]$, y $f'(a+i*h)$ viene dado por:
$$
f'(a+i*h)=\frac{f(a+(i+1)*h) - f(a+i*h)}{h}, \forall i \in \{0,1,...,N\} \tag{5}
$$

# 2.  Método para resolverlo

## 2.1. Breve introducción a la GE

La evolución gramatical codifica un conjunto de números pseudo aleatorios **(codones)** en un cromosoma que consiste en un número variable de genes binarios de 8 bits. 

Estos números se usan para seleccionar una regla apropiada a partir de una definición de gramática con **notación Backus-Naur (BNF)**.

**Una gramática de BNF consiste en la tupla**
$$  \{N, T, P, S \} $$
dónde:
- **N** es el conjunto de no terminales, 
- **T** es el conjunto de terminales
- **P** es un conjunto de reglas de producción que mapean los elementos de N a T 
- **S** es un símbolo de inicio que es un miembro de N. 
  
Los no terminales de la gramática se mapean en los terminales de la gramática mediante la aplicación recursiva de las reglas dictadas por los valores de los genes. Al finalizar el proceso de mapeo, el código final producido (fenotipo) está formado sólo por terminales.

Por ejemplo, si consideramos esta gramática BNF:

```
N = { expr, op, pre_op } 
T = { Sin, Cos, Tan, Log, +, -, /, *, X, () } 
S = <expr>
```

Y representamos P por:

```
(1) <expr> ::= <expr> <op> <expr>     (A) 
             | ( <expr> <op> <expr> ) (B)
             | <pre-op> ( <expr> )    (C) 
             | <var>                  (D)
(2) <op> ::= + (A)  
           | - (B) 
           | / (C) 
           | * (D)
(3) <pre-op> ::= Sin (A)  
               | Cos (B) 
               | Tan (C) 
               | Log (D)
(4) <var> ::= X
```

Consideremos la regla (1):

```
(1) <expr> ::= <expr> <op> <expr> 
             | ( <expr> <op> <expr> ) 
             | <pre-op> ( <expr> ) 
             | <var>
```

En este caso, el no terminal puede producir uno de cuatro resultados diferentes, para decidir cuál utilizar nuestro sistema toma el siguiente número aleatorio disponible del cromosoma y, en este caso obtiene el módulo cuatro del número para decidir qué regal de producción toma. 

Cada vez que se tiene que tomar una decisión, se lee otro número pseudo aleatorio del cromosoma, y de esta manera, el sistema atraviesa el cromosoma.

En GE es posible que los individuos se queden sin genes durante el proceso de mapeo, y en este caso hay dos alternativas:
- La primera es declarar al individuo inválido y castigarlos con un valor de fitness adecuado
- La segunda es envolver al individuo y reutilizar los genes. Este es un enfoque bastante inusual en EAs, ya que es completamente posible que ciertos genes se usen dos o más veces. 

**Lo que es crucial, sin embargo, es que cada vez que un individuo en particular es mapeado de su genotipo a su fenotipo, se genera la misma salida. Esto se garantiza por el proceso de mapeo descrito anteriormente.**

## 2.2. Idoneidad de GE para resolver el problema

Nuestro problema consiste en encontrar la derivada de una función probando/evolucionando distintas combinaciones de otras dadas.

Según leemos en [grammatical-evolution.org](http://www.grammatical-evolution.org/papers/gp98/node1.html): 

> *GE has proved successful when applied to a symbolic regression problem [Ryan 98a], and finding trigonometric identities [Ryan 98b], here we apply GE to a symbolic integration problem taken from the literature [Koza 92]. This involves finding a function which is an integral of Cos(X)+2X+1. In each of these cases we take a subset of C as our target language which is described in Backus Naur Form definition. A Steady State selection mechanism [Syswerda 89] has been employed and was found to reduce the number of generations required to achieve a correct solution. Using this selection mechanism we reapplied our system to the two problems previously tackled and again found an improvement in performance for both of these problems.*

Por lo tanto, según esta referencia, parece que utilizar una técnica evolutiva basada en Evolución Gramatical podría ser la forma más indicada.

Además, teniendo en cuenta que el fundamente de la evolución gramatical se basa en definir una gramática que representa las formas válidas que pueden adoptar los individuos, y teniendo en cuenta que una función objetivo puede alcanzarse como combinación de operadores, números y otras funciones básicas, parece bastante natural que el problema de encontrar la derivadda de una función pueda abordar con técnicas de GE.

# 2.3. Expresión matemática de la función de evaluación

Primero necesitamos una función que nos permita tomar $N$ muestras del valor de la derivada de una función $f$ en un intervalo $D = [a,b]$.

In [318]:
from math import sin, cos, exp, log

def derivada(f, D, N):    
    """
    Calcula los valores aproximados de la derivada
    de f en el intervalo D tomando N muestras    
    """
    a,b = D
    h = float(b-a)/N
    return [(f(a + (i+1)*h) - f(a + i*h))/h 
            for i in range(N+1)]

# derivada(sin, [0,1], 10)

Necesitamos además poder tomar $N$ muestras de la función dada por el fenotipo compilado de un individuo en un intervalo $D=[a,b]$

In [317]:
def muestrea(f, D, N):
    """
    Calcula los valores de la función que se 
    corresponde con el fenotipo de un individuo 
    en el intervalo D tomando N muestras    
    """
    a,b = D
    h = float(b-a)/N
    return [f(a + i*h) for i in range(N+1)]

# muestrea(lambda x: cos(x), [0,1], 10)

Podemos construir además la función que determina el error cometido al considerar un fenotipo $ \text{fen}$ como la derivada de una función $f$.

La función de evaluación viene dada por la expresión (4) indicada en el enunciado de la práctica. Existen diferentes formas de calcular el error; en esta práctica vamos a utilizar el error cuadrático.


In [140]:
def mean_square_error(l,r):
    """
    Calcula el error cuadrático medio
    entre dos listas de números
    """ 
    squares = [(l[i] - r[i])**2 
               for i in range(len(l))]
    return sum(squares)/float(len(l))

# mean_square_error([4,5],[0,0])

Así por ejemplo, para el caso de seno y coseno, podemos hacer esta prueba:

In [240]:
l = derivada(sin, [0,1], 10)
r = muestrea(lambda x: cos(x), [0,1], 10)
print 'Error para sin´ = cos: ', mean_square_error(l,r)

Error para sin´ = cos:  0.0007595619669


Siguiendo las indicaciones del enunciado de la práctica, en lugar de implementar desde cero el código que interpretase la gramática y realizara la decodificación de los individuos, prefiero apoyarme en una librería ya existante, [ponyge](https://github.com/jmmcd/ponyge). 

De ésta extraigo sólo la funcionalidad que permite interpretar la gramática y decodificar un individuo. No utilizo la librería tal cuál está disponible en la red, sino que me quedo sólo con la parte relevante, la cual incluyo en el archivo python ponyge.py. De este modo, puedo tener un objecto que interprete una gramática dada con una simple llamada:

In [115]:
import sys
sys.path.insert(0, 'ponyge.py')

from ponyge import Grammar

G = Grammar("""
<expr>   ::= <expr><op><expr> \
           | (<expr><op><expr>) \
           | <pre_op>(<expr>) \
           | <var>
<op>     ::= + | - | * | / 
<pre_op> ::= sin | cos | exp | log
<var>    ::= x | 1.0
""", MAX_WRAPS=0)

Y puedo obtener el fenotipo de un individuo cualquiera llamando al método **generate** del objeto que contiene la gramática. Nótese que el método generate devuelve:
- el fenotipo del individuo de entrada si puede ser decodificado según la gramática. None en otro caso
- el número de codones que se necesitan para la decodificación

In [116]:
import numpy as np
        
finish = False
while not finish:
    fen = G.generate([np.random.randint(0, 10)
                      for _ in range(5)])
    if fen[0]:
        print fen
        finish = True        

('x', 2)


Por último, haciendo uso de las funciones anteriores, podemos construir la **función de evaluación genérica**, que dadas:
- una función f
- un individuo 
- un intervalo real D=[a,b]
- y un valor N de muestreo en el intervalo

nos devuelva el fitness del individuo, o None si se produce cualquier error en la evaluación.

In [316]:
from math import sin

def mean_sqr_fitness(f, D, N, ind):
    """
    Devuelve el fitness, visto como error cuadrático 
    medio en la aproximación a la derivada de f, 
    del individuo ind en el 
    intervalo D muestrado por N puntos.
    Si se produce cualquier error
    (típicamente divisiones por cero), se devuelve
    None.    
    """
    try:
        l = derivada(f, D, N)
        r = muestrea(ind.compiled_phenotype, D, N)
        return mean_square_error(l,r)
    except:
        return None

# 2.4 Operadores de inicialización, variación y selección

La generación de individuos la llevo a cabo creando listas de números enteros aleatorios en donde puedo indicar el valor máximo de cada codón y el la longitud del genotipo:

In [267]:
import random
from functools import partial

class Individual(object):
    """Clase que representa los individuos"""
    def __init__(self, genome=None, CODON_SIZE=127, 
                 LENGTH=100, PENALTY=1e6):
        if genome == None:
            self.genome = [random.randint(0, CODON_SIZE)
                           for _ in range(LENGTH)]
        else:
            self.genome = genome
        self.phenotype = None
        self.used_codons = 0
        self.compiled_phenotype = None
        self.fitness = 1e10

    def __lt__(self, other):
        return other.fitness < self.fitness

    def __str__(self):
        return ("Individual: " +
                str(self.phenotype) + "; " + str(self.fitness))

    def evaluate(self, G, fitness_f, PENALTY=1e6):
        self.phenotype, self.used_codons = G.generate(self.genome)
        if self.phenotype is None:
            self.fitness = PENALTY
        else :
            self.compiled_phenotype = eval('lambda x:' + self.phenotype)
            v = fitness_f(self)            
            self.fitness = v if v else PENALTY        

Para la selección, la podemos hacer por torneo:

In [319]:
def tournament_selection(population, 
                         GENERATION_SIZE,
                         TOURNAMENT_SIZE=3):
    """Selecciona por torneo. """
    winners = []
    while len(winners) < GENERATION_SIZE:
        competitors = random.sample(population, 
                                    TOURNAMENT_SIZE)
        competitors.sort(reverse=True)
        winners.append(competitors[0])
    return winners

Y por último, para la variación, podemos mutar o cruzar.

Las mutaciones las realizo a nivel de codón, no de bit, permitiendo que se parametrice la probabilidad de mutación:

In [258]:
def mutate(ind, CODON_SIZE, MUTATION_PROBABILITY=.0):
    """Muta un individuo a nivel de codón con
    probabilidad MUTATION_PROBABILITY.
    """
    for i in range(len(ind.genome)):
        if random.random() < MUTATION_PROBABILITY:
            ind.genome[i] = random.randint(0, CODON_SIZE)
    return ind

Y el cruce lo realizo por un único punto. Conviene observar que permito parametrizar que el cruce se realize o no por partes del genoma que pertenecen a la sección del mismo que es relevante en su decodificación.

In [310]:
def onepoint_crossover(p_0, p_1, 
                       CROSSOVER_PROBABILITY=.1,
                       WITHIN_USED=True):
    """Dados dos individuos, crea un hijo cruzándolos
    por punto único. Si WITHIN_USED es True, el cruce
    se produce por partes del genoma usadas en la 
    decodificación de los padres"""    
    c_p_0, c_p_1 = p_0.genome, p_1.genome
    if WITHIN_USED:
        max_p_0, max_p_1 = p_0.used_codons, p_1.used_codons
    else:
        max_p_0, max_p_1 = len(c_p_0), len(c_p_1)
    
    pt_p_0 = random.randint(1, max_p_0)
    pt_p_1 = random.randint(1, max_p_1)    
    if random.random() < CROSSOVER_PROBABILITY:
        c_0 = c_p_0[:pt_p_0] + c_p_1[pt_p_1:]
        c_1 = c_p_1[:pt_p_1] + c_p_0[pt_p_0:]
    else:
        c_0, c_1 = c_p_0[:], c_p_1[:]
        
    return Individual(c_0), Individual(c_1)

Y por último, el método de reemplazo, generacional con elitismo 1 por defecto, aunque es configurable:

In [313]:
import copy

def generational_replacement(new_pop, pop, 
                             GENERATION_SIZE, 
                             ELITE_SIZE=1):
    """Devuelve la nueva población a partir 
    de la actual y la anterior aplicando elitismo
    """
    pop.sort(reverse=True)
    for ind in pop[:ELITE_SIZE]:
        new_pop.append(copy.copy(ind))
    new_pop.sort(reverse=True)
    return new_pop[:GENERATION_SIZE]

# 2.5 Implementación del algoritmo principal y documentación de parámetros

Po último podemos escribir, con todas las funcionalidades anteriores el algoritmo que gobernará nuestro programa de evolución gramátical.

In [129]:
VERBOSE = False
CODON_SIZE = 127
ELITE_SIZE = 1
POPULATION_SIZE = 100  # Población inicial
GENERATION_SIZE = 100
GENERATIONS = 30
MUTATION_PROBABILITY = 0.01
CROSSOVER_PROBABILITY = 0.7
MAX_WRAPS = 0
#GRAMMAR_FILE, FITNESS_FUNCTION = "grammars/hofBoolean.pybnf", EvenNParityFitness(3)
GRAMMAR_FILE, FITNESS_FUNCTION = "grammars/letter.bnf", StringMatch("golden")
#GRAMMAR_FILE, FITNESS_FUNCTION = "grammars/arithmetic.pybnf", MaxFitness()
#GRAMMAR_FILE, FITNESS_FUNCTION = "grammars/boolean.pybnf", XORFitness()


def run(config):
    """Ejecuta el programa de evolución gramatical"""
    
    # Creamos la gramática
    G = Grammar(config['G'], 
                config['MAX_WRAPS']) 
        
    # Población inicial
    pop = [Individual() for _ in range(config['POPULATION_SIZE'])]
    
    #Evaluate initial population
    evaluate_fitness(individuals, grammar, fitness_function)
    best_ever = max(individuals)
    individuals.sort(reverse=True)
    print_stats(1, individuals)
    for generation in range(2, (max_generations+1)):
        individuals, best_ever = step(individuals, grammar, replacement, selection, fitness_function, best_ever)
            """Return individuals and best ever individual from a step of
        the EA iteration"""
        #Select parents
        parents = selection(individuals)
        #Crossover parents and add to the new population
        new_pop = []
        while len(new_pop) < GENERATION_SIZE:
            new_pop.extend(onepoint_crossover(*random.sample(parents, 2)))
        #Mutate the new population
        new_pop = list(map(int_flip_mutation, new_pop))
        #Evaluate the fitness of the new population
        evaluate_fitness(new_pop, grammar, fitness_function)
        #Replace the sorted individuals with the new populations
        individuals = replacement(new_pop, individuals)
        best_ever = max(best_ever, max(individuals))
        return individuals, best_ever
        
        
        print_stats(generation, individuals)
    return best_ever





# f = partial(fitness, sin, [0,1], 10)


# i1 =Individual()
# i2 =Individual()
# i1.evaluate(G,f)
# i2.evaluate(G,f)
# o=onepoint_crossover(i1, i2)
# print o[0]

IndentationError: unexpected indent (<ipython-input-129-3af2379adc73>, line 33)

In [314]:


VERBOSE = False
CODON_SIZE = 127
ELITE_SIZE = 1
POPULATION_SIZE = 100  # Población inicial
GENERATION_SIZE = 100
GENERATIONS = 3
MUTATION_PROBABILITY = 0.01
CROSSOVER_PROBABILITY = 0.7
MAX_WRAPS = 0
#GRAMMAR_FILE, FITNESS_FUNCTION = "grammars/hofBoolean.pybnf", EvenNParityFitness(3)
# GRAMMAR_FILE, FITNESS_FUNCTION = "grammars/letter.bnf", StringMatch("golden")
#GRAMMAR_FILE, FITNESS_FUNCTION = "grammars/arithmetic.pybnf", MaxFitness()
#GRAMMAR_FILE, FITNESS_FUNCTION = "grammars/boolean.pybnf", XORFitness()



# Creamos la gramática
G = Grammar("""
<expr>   ::= <expr><op><expr> \
           | (<expr><op><expr>) \
           | <pre_op>(<expr>) \
           | <var>
<op>     ::= + | - | * | / 
<pre_op> ::= sin | cos | exp | log
<var>    ::= x | 1.0
""", MAX_WRAPS=0)

f = partial(fitness, sin, [0,1], 10)



# Población inicial
pop = [Individual() for _ in range(POPULATION_SIZE)]

# #Evaluate initial population
for ind in pop:
    ind.evaluate(G, f)
    
best = max(pop)

pop.sort(reverse=True)

for generation in range(1, GENERATIONS):
#     pop, best = step(individuals, grammar, replacement, selection, fitness_function, best_ever)
    
    #Select parents
    parents = tournament_selection(pop, GENERATION_SIZE)
     
    #Crossover parents and add to the new population
    new_pop = []
    while len(new_pop) < GENERATION_SIZE:
        new_pop.extend(onepoint_crossover(*random.sample(parents, 2)))
        
    #Mutate the new population
    for ind in new_pop:
        mutate(ind, CODON_SIZE)
    
    #Evaluate the fitness of the new population
    for ind in new_pop:
        ind.evaluate(G, f)
    
#     #Replace the sorted individuals with the new populations
    pop = generational_replacement(new_pop, pop)
#     best_ever = max(best_ever, max(individuals))
#     return individuals, best_ever


#     print_stats(generation, individuals)
# return best_ever